# NSU, vaje 2: meta učenje

### A: Spletna stran OpenML in knjižnica **openml**, priprava podatkov

A.1 Dobi podatke z OpenML-ja (https://www.openml.org/).
Najdi vsa podatkovja, ki imajo med 100 in 200 primerov in med 4 do 100 znacilk. Pomagaj si s funkcijo **openml.datasets.list_datasets**. Podaj ji argumente
- number_instances (npr. number_instances="\<x>..\<y>", kjer sta x in y najmanjse in najvecje dovoljeno stevilo primerov, npr. "10..20")
- number_features (analogno number_instances)
- output_format="dataframe"
in si oglej dobljeni pandasov Dataframe. Eden od stolpcev v njem je did (data ID).

Za opis OpenML funkcij lahko uporabis dokumentacijo na https://docs.openml.org/Python-API/

In [60]:
import numpy as np
import pandas as pd
import openml

d = openml.datasets.list_datasets(number_instances='100..200', number_features='4..100', output_format="dataframe")
print(d)


         did                 name  version uploader  status format  \
10        10                lymph        1        1  active   ARFF   
48        48                  tae        1        1  active   ARFF   
55        55            hepatitis        1        1  active   ARFF   
61        61                 iris        1        1  active   ARFF   
62        62                  zoo        1        1  active   ARFF   
...      ...                  ...      ...      ...     ...    ...   
43839  43839  IRIS-flower-dataset        1    30125  active   arff   
43859  43859            iriiiiiis        2    28351  active   ARFF   
44151  44151                 Iris       51    30495  active   arff   
44154  44154      iris_reproduced        1    30495  active   arff   
44344  44344                 Iris       52    30495  active   arff   

       MajorityClassSize  MaxNominalAttDistinctValues  MinorityClassSize  \
10                  81.0                          8.0                2.0   
48     

A.2 Nalozi ta podatkovja s funkcijo **openml.datasets.get_datasets**, tako da uporabis ID-je podatkovij iz prejsnje tocke. To bo prvic morda trajalo nekaj minut. Ker se bodo nalozena podatkovja shranila tudi na disk racunalnika, bo vsak naslednji klic precej hitrejsi.

In [61]:
ids = list(d["did"])
print(ids)

podatkovja_all = openml.datasets.get_datasets(ids)
print(podatkovja_all)

[10, 48, 55, 61, 62, 164, 187, 191, 195, 199, 203, 206, 207, 210, 213, 222, 229, 232, 285, 327, 328, 329, 338, 434, 444, 446, 448, 449, 461, 463, 492, 497, 501, 506, 520, 521, 526, 535, 551, 585, 587, 591, 594, 600, 611, 621, 624, 625, 629, 630, 634, 636, 639, 640, 642, 651, 655, 656, 665, 678, 714, 716, 719, 721, 726, 736, 738, 745, 747, 748, 753, 754, 756, 762, 768, 771, 775, 783, 784, 788, 789, 808, 812, 829, 850, 852, 854, 868, 875, 876, 878, 885, 889, 890, 902, 916, 921, 922, 932, 941, 944, 955, 956, 965, 969, 973, 974, 1006, 1007, 1012, 1026, 1045, 1054, 1059, 1061, 1064, 1066, 1070, 1075, 1097, 1099, 1115, 1228, 1413, 1441, 1448, 1450, 1455, 1456, 1463, 1465, 1473, 1488, 1490, 1512, 1513, 1519, 1520, 1556, 1559, 4153, 23420, 40669, 40681, 40916, 41477, 41490, 41510, 41511, 41552, 41558, 41567, 41568, 41582, 41583, 41669, 41950, 41952, 41953, 41961, 41962, 41976, 41977, 41978, 41996, 41997, 42002, 42003, 42010, 42011, 42015, 42016, 42020, 42021, 42025, 42026, 42030, 42031, 42035,

A.3 Iz zbirke podatkovij odstrani tista, ki niso primerna za klasifikacijo. Pomagas si lahko s klicem **podatkovje.get_data()**, ki vrne

- x: pandasov Dataframe znacilk, ki vkljucuje tudi ciljno spremenljivko, ce argument target in podan
- y: stolpec, ki podaja vrednosti ciljne spremenljivke, ce je argument target podan, in None sicer
- nominalni: seznam vrednosti True/False, ki pove, ali je i-ti atribut nominalen
- atributi: seznam imen atributov

Ciljno spremenljivko posameznega podatkovja najdemo z **podatkovje.default_target_attribute**.

Iz vseh podatkovij iz prejsnje naloge odstrani tista, ki imajo
- neznano (None) ali numericno ciljno spremenljivko,
- več ciljnih spremenljivk.

Pazi, da obdržiš le eno razlicico podatkov: "iris" se npr. pojavi vec kot 40-krat. Ime podatkovja je shranjeno v
polju "name" (do njega torej dostopamo s "podatkovje.name")
Ker scikit ne podpira nominalnih znacilk, odstrani tudi vsa podatkovja, ki vsebujejo nominalne znacilke.


In [75]:


def precisti_podatkovja(podatkovja):
    podatkovja_ok = {}
    for n, podatkovje in enumerate(podatkovja):
        name_podatkovja = podatkovje.name
        target = podatkovje.default_target_attribute
        X, _, nominal, names = podatkovje.get_data()
        # Vec pogojev, da podatkovje obdrzimo:

        # ce ga nismo ze prej (tj. neke druge verzije)
        name_ok = name_podatkovja not in podatkovja_ok

        # ce ima znan target in je target en sam (in ne npr. "Spol,Starost")
        targ_ok = target is not None and "," not in target

        # ce je target nominalen (klasifikacija) in podatki nimajo nominalnih atributov
        i_target = names.index(target) if targ_ok else 0
        nomi_ok = nominal[i_target] and sum(nominal) == 1  # natanko en nominalen in to je target
        if name_ok and targ_ok and nomi_ok:
            podatkovja_ok[name_podatkovja] = n

    n_vsa = len(podatkovja)
    n_ok = len(podatkovja_ok)
    print(f"Obdrzal sem {n_ok} podatkovij ({100 * n_ok / n_vsa:.1f}%)")
    return podatkovja_ok


podatkovja_ok  = precisti_podatkovja(podatkovja_all)


Obdrzal sem 52 podatkovij (20.6%)


### B - Priprava ciljnih spremenljivk za meta učenje 
Naša ciljna spremenljivka bo uspešnost posameznih metod strojnega učenja na različnih podatkih. 

Na vseh OK podatkovjih pozeni modele kNN (**sklearn.neighbors.KNeighborsClassifier**), odlocitveno drevo (**sklearn.tree.DecisionTreeClassifier**) in naivnega Bayesa (**sklearn.naive_bayes.GaussianNB**).
Za smiselno oceno kvalitete je treba razbiti podakovje na ucno in testno mnozico. V testno mnozico daj 1/4 primerov. 
Kvaliteto napovedi izmeri s tocnostjo.
Ce se bo pri ucenju/napovedovanju koda sesula, to morda pomeni, da s podatki ni vse v redu in bi jih bilo potrebno prej
odfiltrirati.
Dobljene rezultate shrani v tabelo s stolpci

**ime podatkovja,knn,drevo,bayes,najboljsi**

kjer stolpci 2-4 podajajo tocnosti modelov na tesni podmnozici podatkovja, stolpec najboljsi pa ime najboljsega
modela. Pandasov dataframe lahko preprosto shraniš v CSV datoteko s funkcijo **df.to_csv**.

In [81]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

knn = KNeighborsClassifier(15)
tree = DecisionTreeClassifier(random_state=0)
bayes = GaussianNB()

modeli = {"knn": knn, "tree": tree, "bayes": bayes}
data_dict = {"name": [], "knn": [], "tree": [], "bayes": [], "best": []}

for name, n in podatkovja_ok.items():

    podatkovje = podatkovja_all[n]
    target = podatkovje.default_target_attribute
    
    X, y,_,_ = podatkovje.get_data(target=target)
    X = np.array(X)
    y = np.array([str(t) for t in y])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    data_dict["name"].append(name)
    best_acc, best_model = 0, ""
    
    for model_name, model in modeli.items():
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)

        if accuracy > best_acc:
            best_model = model_name
            best_acc = accuracy
        data_dict[model_name].append(accuracy)
    
    data_dict["best"].append(best_model)

meta_df = pd.DataFrame(data=data_dict, index=None)
meta_df.to_csv("meta_target.csv", index=False)


### C Priprava napovednih spremenljivk za meta učenje

Pri pripravi meta znacilk nam bo pomagal paket pymfe https://pypi.org/project/pymfe/.
Najbolj koristno bo orodje **MFE**, ki mu v argumentu **groups** povemo, katere tipe meta značilk želimo.
Po nastavitvi moramo poklicati še njegovi funkciji **fit** in **extract**.

Najprej preizkusimo pymfe na enem izmed podatkovij v naši zbirki.

C.1 Naredi meta značilke iz podatkov (tip značilk "general" in "info-theory"). Oglej si ustvarjene značilke. 
Preizkusiš lahko tudi druge možnosti, npr. "statistical".

Opozorilo, da nekaterih značilk ni mogoče izračunati, ni nič nenavadnega.

In [102]:
from pymfe.mfe import MFE
import random

#izbere nakljucni element v slovarju izbranih podattkov
name, n = random.choice(list(podatkovja_ok.items()))

podatkovje = podatkovja_all[n]
target = podatkovje.default_target_attribute
X, y,_,_ = podatkovje.get_data(target=target)

X = np.array(X)
y = np.array([str(t) for t in y])

# Extract general, statistical and information-theoretic measures
mfe = MFE(groups=["general", "statistical", "info-theory"])
mfe.fit(X, y)
ft = mfe.extract()
print('Imena meta znacilk:', ft[0])
print()
print('Vrednosti meta znacilk za podatkovje:', name, ft[1])




/Users/vitorozman/Documents/School/Master/1.Letnik/ML/Advanced_ML/venv/lib/python3.11/site-packages/pymfe/_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
/Users/vitorozman/Documents/School/Master/1.Letnik/ML/Advanced_ML/venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/vitorozman/Documents/School/Master/1.Letnik/ML/Advanced_ML/venv/lib/python3.11/site-packages/pymfe/_internal.py:731: RuntimeWarning: Can't summarize feature 'can_cor' with summary 'sd'. Will set it as 'np.nan'.
  warnings.warn(
/Users/vitorozman/Documents/School/Master/1.Letnik/ML/Advanced_ML/venv/lib/python3.11/site-packages/pymfe/statistical.py:1272: RuntimeWarning: invalid value encountered in log
  * np.log(n

Imena meta znacilk: ['attr_conc.mean', 'attr_conc.sd', 'attr_ent.mean', 'attr_ent.sd', 'attr_to_inst', 'can_cor.mean', 'can_cor.sd', 'cat_to_num', 'class_conc.mean', 'class_conc.sd', 'class_ent', 'cor.mean', 'cor.sd', 'cov.mean', 'cov.sd', 'eigenvalues.mean', 'eigenvalues.sd', 'eq_num_attr', 'freq_class.mean', 'freq_class.sd', 'g_mean.mean', 'g_mean.sd', 'gravity', 'h_mean.mean', 'h_mean.sd', 'inst_to_attr', 'iq_range.mean', 'iq_range.sd', 'joint_ent.mean', 'joint_ent.sd', 'kurtosis.mean', 'kurtosis.sd', 'lh_trace', 'mad.mean', 'mad.sd', 'max.mean', 'max.sd', 'mean.mean', 'mean.sd', 'median.mean', 'median.sd', 'min.mean', 'min.sd', 'mut_inf.mean', 'mut_inf.sd', 'nr_attr', 'nr_bin', 'nr_cat', 'nr_class', 'nr_cor_attr', 'nr_disc', 'nr_inst', 'nr_norm', 'nr_num', 'nr_outliers', 'ns_ratio', 'num_to_cat', 'p_trace', 'range.mean', 'range.sd', 'roy_root', 'sd.mean', 'sd.sd', 'sd_ratio', 'skewness.mean', 'skewness.sd', 'sparsity.mean', 'sparsity.sd', 't_mean.mean', 't_mean.sd', 'var.mean', 'va

C.2 Naredi meta značilke na podlagi enostavnega modela - odločitvenega drevesa, naučenega na celotni množici. Model ustvarimo in naučimo samo, značilke pa iz naučenega drevesa pridobimo z **MFE** in nastavitvijo "model-based" ter funkcjo **extract_from_model**.

In [106]:
from sklearn.tree import DecisionTreeClassifier


tree_model = DecisionTreeClassifier().fit(X, y)
extractor = MFE(groups=['model-based'])
attribute_names, attribute_values = extractor.extract_from_model(tree_model)

print(list(zip(attribute_names, attribute_values)))

[('leaves', 13), ('leaves_branch.mean', 4.230769230769231), ('leaves_branch.sd', 1.3008872711759818), ('leaves_corrob.mean', 0.0769230769230769), ('leaves_corrob.sd', 0.1468530423021425), ('leaves_homo.mean', 69.99999999999999), ('leaves_homo.sd', 38.248311509572986), ('leaves_per_class.mean', 0.5), ('leaves_per_class.sd', 0.16317848796612636), ('nodes', 12), ('nodes_per_attr', 0.3076923076923077), ('nodes_per_inst', 0.0975609756097561), ('nodes_per_level.mean', 2.0), ('nodes_per_level.sd', 0.8944271909999159), ('nodes_repeated.mean', 1.0909090909090908), ('nodes_repeated.sd', 0.30151134457776363), ('tree_depth.mean', 3.44), ('tree_depth.sd', 1.6093476939431082), ('tree_imbalance.mean', 0.15888394457504434), ('tree_imbalance.sd', 0.12956672693349294), ('tree_shape.mean', 0.24759615384615385), ('tree_shape.sd', 0.13469048385950544), ('var_importance.mean', 0.02564102564102564), ('var_importance.sd', 0.061885317015783384)]


C.3 Izračunaj vse tri vrste značilk za vsa podatkovja v naši zbirki in jih shrani v tabelo. Naredi tabelo s stolpci

**name,znacilka1,znacilka2,...,znacilkaN**, 

kjer je N stevilo znacilk (imena stolpcev 2-N niso znacilkaI, ampak dejanska imena znacilk, ki se ustvarijo.)
Tabela lahko vsebuje manjkajoce vrednosti. Tabelo shrani tudi v datoteko.

In [107]:
meta_features = {"name": []}
for ime_podatkovja, n in podatkovja_ok.items():
    x, y, nominal, names = podatkovja_all[n].get_data(target=podatkovja_all[n].default_target_attribute)
    x = np.array(x)
    y = np.array([str(t) for t in y])  # Ker je prej y tu in tam vseboval True in False

    mfe1 = MFE(groups=["general", "info-theory"]) 
    mfe2 = MFE(groups=["model-based"])
    mfe1.fit(x, y)
    attribute_names, attribute_values = mfe1.extract()
    tree = DecisionTreeClassifier()
    tree.fit(x, y)
    attribute_names2, attribute_values2 = mfe2.extract_from_model(tree)
    attribute_names += attribute_names2
    attribute_values += attribute_values2
    
    is_first = len(meta_features) == 1
    meta_features["name"].append(ime_podatkovja)
    for a_name, a_value in zip(attribute_names, attribute_values):
        if is_first:
            meta_features[a_name] = [a_value]
        else:
            meta_features[a_name].append(a_value)
pd.DataFrame(data=meta_features, index=None).to_csv("meta_features.csv", index=False)

print(meta_features)

/Users/vitorozman/Documents/School/Master/1.Letnik/ML/Advanced_ML/venv/lib/python3.11/site-packages/pymfe/_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
/Users/vitorozman/Documents/School/Master/1.Letnik/ML/Advanced_ML/venv/lib/python3.11/site-packages/pymfe/_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
/Users/vitorozman/Documents/School/Master/1.Letnik/ML/Advanced_ML/venv/lib/python3.11/site-packages/pymfe/_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
/Users/vitorozman/Documents/School/Master/1.Letnik/ML/Advanced_ML/venv/lib/python3.11/site-packages/pymfe/_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
/Users/vitor

{'name': ['iris', 'zoo', 'wine', 'hayes-roth', 'fri_c3_100_50', 'pwLinear', 'fri_c2_100_5', 'visualizing_environmental', 'wisconsin', 'fri_c0_100_5', 'autoPrice', 'fri_c2_100_10', 'fri_c3_100_25', 'fri_c2_100_25', 'fri_c3_100_10', 'triazines', 'fri_c1_100_10', 'fri_c0_100_10', 'fri_c1_100_25', 'fri_c1_100_5', 'fri_c0_100_50', 'fri_c4_100_25', 'fri_c1_100_50', 'fri_c4_100_10', 'transplant', 'fri_c0_100_25', 'fri_c3_100_5', 'fri_c2_100_50', 'fri_c4_100_50', 'kc1-top5', 'kc1-binary', 'datatrieve', 'MyIris', 'KungChi3', 'KnuggetChase3', 'MindCave2', 'breast-tissue', 'fertility', 'parkinsons', 'planning-relax', 'heart-long-beach', 'heart-switzerland', 'robot-failures-lp4', 'robot-failures-lp5', 'Smartphone-Based_Recognition_of_Human_Activities', 'iris_test_upload', 'TuningSVMs', 'JuanFeldmanIris', 'iris-example', 'iriiiiiis', 'iris_reproduced', 'Iris'], 'attr_conc.mean': [0.20922243392447848, 0.122833904281653, 0.07639227680378616, 0.03606470903270312, 0.0319352984987881, 0.0096963892042128